In [1]:
import folium
import gradio as gr

def generate_map(lat, lon):
    print(f"lat: {lat}, lon: {lon}")
    
    # 创建地图，中心点为输入的坐标
    m = folium.Map(
        location=[lat, lon],
        zoom_start=15,  # 设置初始缩放级别
        tiles='OpenStreetMap'  # 使用 OpenStreetMap 底图
    )
    
    # 添加标记
    folium.Marker(
        location=[lat, lon],
        popup=f"位置: {lat}, {lon}",  # 点击标记时显示的信息
        icon=folium.Icon(color='red')  # 设置标记样式
    ).add_to(m)
    
    # 保存到临时HTML文件
    map_html = m._repr_html_()
    return map_html

# 创建 Gradio 接口
interface = gr.Interface(
    fn=generate_map,
    inputs=[
        gr.Number(label="Latitude", value=31.3385, precision=8),
        gr.Number(label="Longitude", value=121.5020, precision=8)
    ],
    outputs=gr.HTML(),  # 使用 HTML 输出
    cache_examples=False
)

interface.launch()

/opt/anaconda3/envs/swarm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


lat: 31.3385, lon: 121.502


: 